# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sat Sep 18 18:53:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:2D:00.0 Off |                  N/A |
| N/A   56C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2019년 Hyundai Data**

In [5]:
lexicon_2019 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2019_hyundai.csv')

lexicon_2019.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,현대차,5380,매일경제,2019010104,[국내 업종별 전망 / 자동차] 내수도 수출도 암울…6대 생산국 지위 흔들,http://news.mk.co.kr/newsRead.php?year=2019&no...,◆ 2019 경제기상도 ◆ \r\n\n\n 지난해 국내 자동차 업계는 한국GM의 군...,2019-01-02,4,117500,118000,113500,114000,565493,-0.037975,-1,-1,경제 기상도 지난해 국내 자동차 업계 한국 군산 공장 철수 시작 가쁜 한국 사태 지...


In [6]:
lexicon_2019[lexicon_2019['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2019.dropna(axis=0, inplace=True)
lexicon_2019 = lexicon_2019.reset_index(drop=True)
lexicon_2019.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
neg_lexicon_2019 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_neg_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_neg와의 Cosine Similarity 계산
for x in range(len(lexicon_2019['Tokenization'])):
    news_num = x+1
    for y in range(len(negative)):
        for z in range(len(list(set(lexicon_2019['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(negative[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(negative[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2019['Tokenization'][x].split())):
                    if lexicon_2019['Tokenization'][x].split()[w] == list(set(lexicon_2019['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2019['date'][x],
                    'news_num': news_num,
                    'KOSELF_neg_word': negative[y],
                    'news_word': list(set(lexicon_2019['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(negative[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                neg_lexicon_2019 = neg_lexicon_2019.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2019['date'][x], negative[y], list(set(lexicon_2019['Tokenization'][x].split()))[z], ko_model.wv.similarity(negative[y], list(set(lexicon_2019['Tokenization'][x].split()))[z])))

***2019-01-02 Cosine Similarity between <둔화> & <악화> : 0.5065980553627014
***2019-01-02 Cosine Similarity between <둔화> & <증가> : 0.5041859149932861
***2019-01-02 Cosine Similarity between <둔화> & <감소> : 0.5349085330963135
***2019-01-02 Cosine Similarity between <약화> & <악화> : 0.5064777731895447
***2019-01-02 Cosine Similarity between <약화> & <감소> : 0.551184892654419
***2019-01-02 Cosine Similarity between <침체> & <경기> : 0.5360627174377441
***2019-01-02 Cosine Similarity between <침체> & <위기> : 0.5072819590568542
***2019-01-02 Cosine Similarity between <하락> & <증가> : 0.553473711013794
***2019-01-02 Cosine Similarity between <하락> & <상승> : 0.7126480340957642
***2019-01-02 Cosine Similarity between <하락> & <감소> : 0.575989305973053
***2019-01-02 Cosine Similarity between <과적> & <차량> : 0.5693626403808594
***2019-01-02 Cosine Similarity between <과적> & <차량> : 0.5693626403808594
***2019-01-02 Cosine Similarity between <과적> & <차량> : 0.5693626403808594
***2019-01-02 Cosine Similarity between <실패> & <실패> : 

In [11]:
neg_lexicon_2019

,date,news_num,KOSELF_neg_word,news_word,cosine_similarity,frequency
0,2019-01-02,1,둔화,악화,0.506598,1
1,2019-01-02,1,둔화,증가,0.504186,2
2,2019-01-02,1,둔화,감소,0.534909,6
3,2019-01-02,1,약화,악화,0.506478,1
4,2019-01-02,1,약화,감소,0.551185,6
...,...,...,...,...,...,...
13526,2020-01-02,3900,하락,상승,0.712648,2
13527,2020-01-02,3901,과적,차량,0.569363,1
13528,2020-01-02,3902,과적,차량,0.569363,1
13529,2020-01-02,3902,하락,급등,0.614909,1


In [12]:
neg_lexicon_2019.to_csv('../../../../Code/Data/Test/Stock-Year/neg_hyundai_2019.csv', index=False)

In [13]:
a = list(set(list(neg_lexicon_2019['news_word'])))

a

['긍적적',
 '폭등',
 '참패',
 '불안',
 '우발',
 '탑차',
 '경기',
 '토히르',
 '악화',
 '급락',
 '특혜',
 '스캔들',
 '유발',
 '강점',
 '무마',
 '보상',
 '엄벌',
 '등락',
 '끝내',
 '강화',
 '선박',
 '과속',
 '급등',
 '손해배상',
 '불황',
 '반등',
 '의심',
 '머락비',
 '추징',
 '작태',
 '경색',
 '결항',
 '차량',
 '연루',
 '플러스',
 '외압',
 '조처',
 '실패',
 '항행',
 '누유',
 '위축',
 '폭로',
 '보합',
 '악재',
 '침체',
 '필루즈',
 '좌절',
 '스테빌리티',
 '폐업',
 '약세',
 '견조',
 '채증',
 '레졸루',
 '증가',
 '급회전',
 '외력',
 '손괴',
 '금수',
 '성공',
 '역풍',
 '부도',
 '약화',
 '하락',
 '쇠퇴',
 '혼돈',
 '둔화',
 '호황',
 '구난',
 '갓길',
 '업황',
 '주춤',
 '상승',
 '저하',
 '조아연',
 '청탁',
 '둥펑샤오캉',
 '비리',
 '감소',
 '화물',
 '논란',
 '폭락',
 '급감',
 '급유',
 '운항',
 '위기',
 '질책',
 '호전',
 '증차',
 '인핏손보']

In [14]:
b = list(set(list(neg_lexicon_2019[neg_lexicon_2019['cosine_similarity']>=0.7]['news_word'])))

b

['의심', '성공', '약화', '스캔들', '실패', '침체', '상승']

In [15]:
c = list(set(list(neg_lexicon_2019[neg_lexicon_2019['cosine_similarity']>=0.65]['news_word'])))

c

['긍적적', '의심', '성공', '급락', '약화', '조처', '스캔들', '실패', '침체', '상승']